In [1]:
import pandas as pd

In [2]:
dados2019 = pd.read_csv('dados/despesa_ceaps_2019.csv', sep=';', skiprows=1, encoding='latin1')
dados2020 = pd.read_csv('dados/despesa_ceaps_2020.csv', sep=';', skiprows=1, encoding='latin1')
dados2021 = pd.read_csv('dados/despesa_ceaps_2021.csv', sep=';', skiprows=1, encoding='latin1')
dados2022 = pd.read_csv('dados/despesa_ceaps_2022.csv', sep=';', skiprows=1, encoding='latin1')
dados = pd.concat([dados2019, dados2020, dados2021, dados2022])
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62828 entries, 0 to 10276
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                62828 non-null  int64 
 1   MES                62828 non-null  int64 
 2   SENADOR            62828 non-null  object
 3   TIPO_DESPESA       62828 non-null  object
 4   CNPJ_CPF           62828 non-null  object
 5   FORNECEDOR         62828 non-null  object
 6   DOCUMENTO          60236 non-null  object
 7   DATA               62828 non-null  object
 8   DETALHAMENTO       34957 non-null  object
 9   VALOR_REEMBOLSADO  62828 non-null  object
 10  COD_DOCUMENTO      62828 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 5.8+ MB


In [3]:
# Conversão do VALOR_REEMBOLSADO para float:
dados['VALOR_REEMBOLSADO'] = dados['VALOR_REEMBOLSADO'].astype('str').str.replace(',', '.').astype('float')

In [4]:
# Remoção de Reembolsos com DATA inválida e conversão de data de string DD/MM/AAAA para datetime:
dados.drop((dados[dados['DATA'].str.match('^(0[1-9]|[12][0-9]|3[01])\/(0[1-9]|1[012])\/[12][0-9]{3}$').astype(bool) == False]).index, inplace=True, axis='index')
dados['DATA'] = pd.to_datetime(dados['DATA'], format='%d/%m/%Y')

In [5]:
# Reembolsos com CPFs ou CNPJs com formatos inválidos:
dados[dados['CNPJ_CPF'].str.match('([0-9]{2}[\.]?[0-9]{3}[\.]?[0-9]{3}[\/]?[0-9]{4}[-]?[0-9]{2})|([0-9]{3}[\.]?[0-9]{3}[\.]?[0-9]{3}[-]?[0-9]{2})') == False].shape[0]

0

In [6]:
# Adicionando nova coluna FORNECEDOR_PESSOA_JURIDICA que indica se o fornecedor referente ao reembolso é pessoa física ou jurídica.
dados['FORNECEDOR_PESSOA_JURIDICA'] = dados['CNPJ_CPF'].str.match('([0-9]{2}[\.]?[0-9]{3}[\.]?[0-9]{3}[\/]?[0-9]{4}[-]?[0-9]{2})')
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,2019-01-04,Despesa com pagamento de energia elétrica do e...,66.02,2116543,True
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,2019-01-04,Despesa com pagamento de energia elétrica do e...,139.98,2116546,True
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,2019-01-07,Despesa com pagamento de aluguel de imóvel par...,6000.00,2113817,False
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,2018-12-25,Despesa com pagamento de telefonia para o escr...,316.39,2116541,True
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,2019-02-04,Despesa com pagamento de energia elétrica para...,99.45,2116550,True
...,...,...,...,...,...,...,...,...,...,...,...,...
10272,2022,7,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,FRMXSO,2022-07-26,"Companhia Aérea: GOL, Localizador: FRMXSO. Pas...",1826.25,2184750,True
10273,2022,7,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,YQRYAW,2022-07-26,"Companhia Aérea: LATAM, Localizador: YQRYAW. P...",2189.41,2184748,True
10274,2022,8,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",27.522.923/0001-60,Casarao da Vila Restaurante LTDA - ME,000127032,2022-08-03,NaN,26.57,2184892,True
10275,2022,8,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,FLXNAO,2022-08-02,"Companhia Aérea: GOL, Localizador: FLXNAO. Pas...",3140.46,2185426,True


In [7]:
fornecedores = dados.groupby(['CNPJ_CPF']).first()['FORNECEDOR']
fornecedores

CNPJ_CPF
00.001.388/0002-26               Distribuidora Brasilia de Veículos S/A
00.005.892/0001-13                                  ML SOUZA E CIA LTDA
00.012.211/0002-25             Drive Car Tansportes e Combustíveis Ltda
00.031.708/0001-00                                            SINPETAXI
00.031.708/0003-71                                            Sinpetaxi
                                            ...                        
976.555.463-04                           Horácio Alves de Andrade  Neto
98.042.294/0001-19                       POSTO DE SERVIÇOS FRITSCH LTDA
98.102.270/0001-08      Comercial de Combustíveis e Cereais Diesel Ltda
98.587.918/0001-83    POSTO DE ABASTECIMENTO DE COMBUSTIVEIS SCHWITZ...
991.568.574-20                                izolda freitas reis pires
Name: FORNECEDOR, Length: 7155, dtype: object

In [8]:
dados = dados.merge(fornecedores, left_on='CNPJ_CPF', right_index=True)
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR_x,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA,FORNECEDOR_y
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,2019-01-04,Despesa com pagamento de energia elétrica do e...,66.02,2116543,True,ENERGISA
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,2019-01-04,Despesa com pagamento de energia elétrica do e...,139.98,2116546,True,ENERGISA
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,2019-02-04,Despesa com pagamento de energia elétrica para...,99.45,2116550,True,ENERGISA
5,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236031,2019-02-04,Despesa com pagamento de energia elétrica para...,262.93,2116551,True,ENERGISA
8,2019,3,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007887449,2019-03-01,Pagamento de energia elétrica para o escritóri...,85.50,2116572,True,ENERGISA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,2022,5,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",31.360.727/0001-03,TRES CASTROS LTDA.,1527,2022-05-04,NaN,327.00,2180809,True,TRES CASTROS LTDA.
10249,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",21.387.460/0004-37,AUTO POSTO ROMA,113802,2022-06-19,NaN,584.02,2183891,True,AUTO POSTO ROMA
10250,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",05.632.683/0001-13,AUTO POSTO TRACYANNE LTDA.,141283,2022-06-03,NaN,497.69,2183889,True,AUTO POSTO TRACYANNE LTDA.
10251,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",04.190.866/0001-63,LUCILENE DOTTO NICOLODI EIRELI,84449,2022-06-18,NaN,400.07,2183890,True,LUCILENE DOTTO NICOLODI EIRELI


In [9]:
dados['FORNECEDOR'] = dados['FORNECEDOR_y']
dados.drop(columns=['FORNECEDOR_x', 'FORNECEDOR_y'], inplace= True)
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA,FORNECEDOR
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,006582758,2019-01-04,Despesa com pagamento de energia elétrica do e...,66.02,2116543,True,ENERGISA
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,006582755,2019-01-04,Despesa com pagamento de energia elétrica do e...,139.98,2116546,True,ENERGISA
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,007236036,2019-02-04,Despesa com pagamento de energia elétrica para...,99.45,2116550,True,ENERGISA
5,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,007236031,2019-02-04,Despesa com pagamento de energia elétrica para...,262.93,2116551,True,ENERGISA
8,2019,3,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,007887449,2019-03-01,Pagamento de energia elétrica para o escritóri...,85.50,2116572,True,ENERGISA
...,...,...,...,...,...,...,...,...,...,...,...,...
10229,2022,5,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",31.360.727/0001-03,1527,2022-05-04,NaN,327.00,2180809,True,TRES CASTROS LTDA.
10249,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",21.387.460/0004-37,113802,2022-06-19,NaN,584.02,2183891,True,AUTO POSTO ROMA
10250,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",05.632.683/0001-13,141283,2022-06-03,NaN,497.69,2183889,True,AUTO POSTO TRACYANNE LTDA.
10251,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",04.190.866/0001-63,84449,2022-06-18,NaN,400.07,2183890,True,LUCILENE DOTTO NICOLODI EIRELI


In [10]:
# Há um número expressivo de reembolsos sem identificação do documento de comprovação:
# Ideias para análises posteriores: alguma correção entre ausência do documento e fraude? Ou (Ausência do documento, Fornecedor é pessoa física (CPF)) e fraude?
dados[dados['DOCUMENTO'].isnull()].shape[0]

2592

In [11]:
#Tipos de Despesas
dados.value_counts(['TIPO_DESPESA']).round(2)

TIPO_DESPESA                                                                                                                                                                                   
Locomoção, hospedagem, alimentação, combustíveis e lubrificantes                                                                                                                                   22957
Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.                                                                                                           13837
Passagens aéreas, aquáticas e terrestres nacionais                                                                                                                                                 13596
Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.      4498
Divu

In [12]:
# Adicionando nova coluna ATRASO_SOLIC_REEMBOLSO com a quantidade de meses que o senador demorou para solicitar o reembolso, a partir da data do documento.
# Ideias para análises posteriores: alguma correção entre maior atraso e fraude?
dados['DIA'] = 1
data_reembolso = pd.to_datetime(dados[['ANO', 'MES', 'DIA']].rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'}))
dados['ATRASO_SOLIC_REEMBOLSO'] = (data_reembolso.dt.year - dados['DATA'].dt.year)*12 + (data_reembolso.dt.month - dados['DATA'].dt.month)
dados[dados['ATRASO_SOLIC_REEMBOLSO'] < 0]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA,FORNECEDOR,DIA,ATRASO_SOLIC_REEMBOLSO
9113,2020,12,MARCOS ROGÉRIO,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,S/N,2021-01-13,NaN,191.69,2154158,True,ENERGISA,1,-1
10956,2021,4,MARCOS ROGÉRIO,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,S/N,2021-05-13,NaN,172.41,2160325,True,ENERGISA,1,-1
13265,2019,5,MARCIO BITTAR,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,176770375,2019-06-03,VALOR PAGO REFRENTE AO SERVIÇO DE TELEFONIA CE...,132.49,2125889,True,OI MÓVEL S.A.,1,-1
2271,2020,10,CONFÚCIO MOURA,Divulgação da atividade parlamentar,13.279.434/0001-14,466,2020-11-10,Despesa com divulgação da atividade parlamentar,400.00,2151319,True,E NASCIMENTO AGÊNCIA DE PUBLICIDADE E MARKETIN...,1,-1
2628,2021,4,CONFÚCIO MOURA,Divulgação da atividade parlamentar,13.279.434/0001-14,495,2021-05-03,Despesa com divulgação da atividade parlamentar,400.00,2159613,True,E NASCIMENTO AGÊNCIA DE PUBLICIDADE E MARKETIN...,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8090,2022,2,RENAN CALHEIROS,"Contratação de consultorias, assessorias, pesq...",35.784.291/0001-30,4,2022-03-29,Prestação de serviços de elaboração de vídeos ...,3000.00,2177255,True,WALDEMBERGUE DA SILVA DANTAS PEREIRA,1,-1
8107,2022,3,RENAN CALHEIROS,"Contratação de consultorias, assessorias, pesq...",38.270.916/0001-34,3,2022-04-25,"Prestação de serviços de elaboração de vídeos,...",1400.00,2178854,True,FILIPE JOSE ALBUQUERQUE CORDEIRO,1,-1
8254,2022,2,RODRIGO CUNHA,"Contratação de consultorias, assessorias, pesq...",33.617.597/0001-77,3,2022-03-04,DESPESA DE CONSULTORIA EM MKT DIGITAL DO SENAD...,1670.00,2177292,True,PRISCILA TENORIO TOLEDO PEIXOTO,1,-1
9000,2022,1,SORAYA THRONICKE,Aquisição de material de consumo para uso no e...,32.887.404/0001-35,00005868,2022-02-04,NaN,150.00,2174152,True,Denis Freire Rocha Tecnologia em informação ME,1,-1


In [13]:
dados[dados['ATRASO_SOLIC_REEMBOLSO'] > 200]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA,FORNECEDOR,DIA,ATRASO_SOLIC_REEMBOLSO
7777,2022,3,PAULO ROCHA,"Passagens aéreas, aquáticas e terrestres nacio...",16.978.175/0001-08,GENXJE,2002-03-16,"Companhia Aérea: LATAM, Localizador: GENXJE. P...",3553.23,2176808,True,ADRIA VIAGENS E TURISMO LTDA ME,1,240
7747,2022,2,PAULO ROCHA,"Aluguel de imóveis para escritório político, c...",02.558.157/0001-62,1129495338,2002-02-11,NaN,1106.13,2174871,True,VIVO - TELEFÔNICA BRASIL S/A,1,240
7829,2022,7,PAULO ROCHA,"Aluguel de imóveis para escritório político, c...",04.895.728/0001-80,0202206014025948,2002-06-21,NaN,1605.00,2183727,True,CELPA CENTRAIS ELÉTRICAS DO PARÁ S/A,1,241
12347,2020,3,TASSO JEREISSATI,"Contratação de consultorias, assessorias, pesq...",20.647.510/0001-62,00000061,2002-03-14,NaN,14100.00,2143467,True,FAAV CONSULTORIA ECONOMICA E EMPRESARIAL LTDA,1,216
208,2021,2,ALESSANDRO VIEIRA,"Passagens aéreas, aquáticas e terrestres nacio...",39.827.859/0001-04,LNXEJH,2002-02-19,"Companhia Aérea: LATAM, Localizador: LNXEJH. P...",2666.43,2156738,True,MM TOUR,1,228
8466,2022,2,ROGÉRIO CARVALHO,"Passagens aéreas, aquáticas e terrestres nacio...",34.848.898/0001-74,SRPSJX,2002-02-02,"Companhia Aérea: LATAM, Localizador: SRPSJX. P...",2260.60,2174993,True,Peakalika Turismo,1,240
7751,2022,2,PAULO ROCHA,"Contratação de consultorias, assessorias, pesq...",34.527.666/0001-14,17,2002-03-22,NaN,8000.00,2176739,True,Nailson Santos Guimarães 12404195204,1,239


In [14]:
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA,FORNECEDOR,DIA,ATRASO_SOLIC_REEMBOLSO
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,006582758,2019-01-04,Despesa com pagamento de energia elétrica do e...,66.02,2116543,True,ENERGISA,1,0
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,006582755,2019-01-04,Despesa com pagamento de energia elétrica do e...,139.98,2116546,True,ENERGISA,1,0
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,007236036,2019-02-04,Despesa com pagamento de energia elétrica para...,99.45,2116550,True,ENERGISA,1,0
5,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,007236031,2019-02-04,Despesa com pagamento de energia elétrica para...,262.93,2116551,True,ENERGISA,1,0
8,2019,3,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,007887449,2019-03-01,Pagamento de energia elétrica para o escritóri...,85.50,2116572,True,ENERGISA,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,2022,5,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",31.360.727/0001-03,1527,2022-05-04,NaN,327.00,2180809,True,TRES CASTROS LTDA.,1,0
10249,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",21.387.460/0004-37,113802,2022-06-19,NaN,584.02,2183891,True,AUTO POSTO ROMA,1,0
10250,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",05.632.683/0001-13,141283,2022-06-03,NaN,497.69,2183889,True,AUTO POSTO TRACYANNE LTDA.,1,0
10251,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",04.190.866/0001-63,84449,2022-06-18,NaN,400.07,2183890,True,LUCILENE DOTTO NICOLODI EIRELI,1,0


In [21]:
dados.describe().round(2)
dados[dados['VALOR_REEMBOLSADO'] == 120000]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,FORNECEDOR_PESSOA_JURIDICA,FORNECEDOR,DIA,ATRASO_SOLIC_REEMBOLSO
13262,2020,12,WELLINGTON FAGUNDES,"Contratação de consultorias, assessorias, pesq...",23.721.017/0001-33,000.000.765,2020-12-21,Despesa com serviço de consultoria e assessori...,120000.0,2153883,True,INCINE VÍDEO E PRODUÇÕES,1,0
